In [ ]:
#from google.colab import drive
#drive.mount("/content/gdrive", force_remount = True)

In [ ]:
#%cd '/content/gdrive/My Drive/LDS8_DeepLearning/LDS8_DinhVietHa_thi/'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
data = pd.read_csv("./kddcup.data.corrected", header=None)
data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
5,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.
6,0,tcp,http,SF,235,1337,0,0,0,0,...,6,1.0,0.0,0.17,0.0,0.0,0.0,0.0,0.0,normal.
7,0,tcp,http,SF,234,1364,0,0,0,0,...,7,1.0,0.0,0.14,0.0,0.0,0.0,0.0,0.0,normal.
8,0,tcp,http,SF,239,1295,0,0,0,0,...,8,1.0,0.0,0.12,0.0,0.0,0.0,0.0,0.0,normal.
9,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 42 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       int64  
 1   1       object 
 2   2       object 
 3   3       object 
 4   4       int64  
 5   5       int64  
 6   6       int64  
 7   7       int64  
 8   8       int64  
 9   9       int64  
 10  10      int64  
 11  11      int64  
 12  12      int64  
 13  13      int64  
 14  14      int64  
 15  15      int64  
 16  16      int64  
 17  17      int64  
 18  18      int64  
 19  19      int64  
 20  20      int64  
 21  21      int64  
 22  22      int64  
 23  23      int64  
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      int64  
 32  32      int64  
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64
 40  

In [ ]:
X = data.iloc[:, :41]
y = data.iloc[:,-1]

In [ ]:
y.unique()

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

In [ ]:
print(type(y))

<class 'pandas.core.series.Series'>


In [ ]:
len(y)

4898431

In [ ]:
abnormal = ['buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.']

In [ ]:
for text in abnormal: 
  y = y.str.replace(text,'others')

In [ ]:
[y == "rootkit."] == True

False

In [ ]:
unique_y, counts_y = np.unique(y, return_counts=True)

In [ ]:
display(unique_y,counts_y)

array(['normal.', 'others'], dtype=object)

array([ 972781, 3925650], dtype=int64)

The number of abnormal intrusions outnumbered that of normal intrusions by almost 4 times.

### Data Preprocessing

Scaler

In [ ]:
X_scale = X.iloc[:,4:6]

In [ ]:
X_scale.describe()

,4,5
count,4.898431e+06,4.898431e+06
mean,1.834621e+03,1.093623e+03
std,9.414311e+05,6.450123e+05
min,0.000000e+00,0.000000e+00
25%,4.500000e+01,0.000000e+00
50%,5.200000e+02,0.000000e+00
75%,1.032000e+03,0.000000e+00
max,1.379964e+09,1.309937e+09


In [ ]:
X.iloc[:,0].unique()

array([   0,   22,   60, ..., 2906, 2911, 2966], dtype=int64)

In [ ]:
X.iloc[:,1].unique()

array(['tcp', 'udp', 'icmp'], dtype=object)

In [ ]:
X.iloc[:,2].unique()

array(['http', 'smtp', 'domain_u', 'auth', 'finger', 'telnet', 'eco_i',
       'ftp', 'ntp_u', 'ecr_i', 'other', 'urp_i', 'private', 'pop_3',
       'ftp_data', 'netstat', 'daytime', 'ssh', 'echo', 'time', 'name',
       'whois', 'domain', 'mtp', 'gopher', 'remote_job', 'rje', 'ctf',
       'supdup', 'link', 'systat', 'discard', 'X11', 'shell', 'login',
       'imap4', 'nntp', 'uucp', 'pm_dump', 'IRC', 'Z39_50', 'netbios_dgm',
       'ldap', 'sunrpc', 'courier', 'exec', 'bgp', 'csnet_ns', 'http_443',
       'klogin', 'printer', 'netbios_ssn', 'pop_2', 'nnsp', 'efs',
       'hostnames', 'uucp_path', 'sql_net', 'vmnet', 'iso_tsap',
       'netbios_ns', 'kshell', 'urh_i', 'http_2784', 'harvest', 'aol',
       'tftp_u', 'http_8001', 'tim_i', 'red_i'], dtype=object)

In [ ]:
X.iloc[:,3].unique()

array(['SF', 'S2', 'S1', 'S3', 'OTH', 'REJ', 'RSTO', 'S0', 'RSTR',
       'RSTOS0', 'SH'], dtype=object)

Encoder

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
X = X.values
y = y.values

In [ ]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:,2])
labelencoder_X_3 = LabelEncoder()
X[:, 3] = labelencoder_X_3.fit_transform(X[:,3])

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X.tolist())

In [ ]:
import pickle

label1 = "label_encode_X_1.pkl"
pickle.dump(labelencoder_X_1, open(label1, "wb"))
label2 = "label_encode_X_2.pkl"
pickle.dump(labelencoder_X_2, open(label2, "wb"))
label3 = "label_encode_X_3.pkl"
pickle.dump(labelencoder_X_3, open(label3, "wb"))

In [ ]:
X[:10]

array([[0.0, 1.0, 0.0, 0, 24, 9, 215, 45076, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0,
        0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0, 24, 9, 162, 4528, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 2, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1, 1,
        1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0, 24, 9, 236, 1228, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2, 2,
        1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0, 24, 9, 233, 2032, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 2, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3, 3,
        1.0, 0.0, 0.33, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0, 24, 9, 239, 486, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 3, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 4, 4,
        1.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 

In [ ]:
X = X[:,1:]

In [ ]:
X[:10]

array([[1.0, 0.0, 0, 24, 9, 215, 45076, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0, 0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0, 24, 9, 162, 4528, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 2, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1, 1,
        1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0, 24, 9, 236, 1228, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2, 2,
        1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0, 24, 9, 233, 2032, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 2, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3, 3,
        1.0, 0.0, 0.33, 0.0, 0.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0, 24, 9, 239, 486, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 3, 3, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 4, 4, 1.0,
        0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0, 24, 9, 238, 1282, 0, 0

In [ ]:
X.shape

(4898431, 42)

In [ ]:
X = np.asarray(X).astype('float32')

In [ ]:
y[:5]

array([0, 0, 0, 0, 0])

In [ ]:
mmc = MinMaxScaler()
X_train = mmc.fit_transform(X)

scalerfile = "mmc.sav"
pickle.dump(mmc, open(scalerfile, 'wb'))

In [ ]:
X_train[:5]

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.4782609e-01,
        9.0000004e-01, 1.5580117e-07, 3.4410805e-05, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.9569471e-03,
        1.9569471e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.4782609e-01,
        9.0000004e-01, 1.1739437e-07, 3.4566535e-06, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0

Build Model

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Dense(units = 12, input_shape=(X.shape[1],), activation='relu'))
classifier.add(Dense(units = 8, activation='relu'))
classifier.add(Dense(units = 1, activation='sigmoid'))

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy',\
                   metrics = ['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                516       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 629
Trainable params: 629
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early = EarlyStopping(patience=10)

checkpoint = ModelCheckpoint('Q2_KDD_checkpoint_new.h5', save_best_only=True)

In [ ]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
print(t0)

2020-11-29 03:02:32.936584


In [ ]:
history = classifier.fit(X_train, y, epochs=100, 
                    batch_size=32, 
                    verbose=1, 
                    validation_split=0.2,
                    callbacks = [early,checkpoint])

Epoch 1/100
122461/122461 [==============================] - 130s 1ms/step - loss: 0.0044 - accuracy: 0.9988 - val_loss: 0.0077 - val_accuracy: 0.9984
Epoch 2/100
122461/122461 [==============================] - 127s 1ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.0070 - val_accuracy: 0.9986
Epoch 3/100
122461/122461 [==============================] - 124s 1ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.0098 - val_accuracy: 0.9987
Epoch 4/100
122461/122461 [==============================] - 107s 876us/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0066 - val_accuracy: 0.9985
Epoch 5/100
122461/122461 [==============================] - 119s 974us/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.0081 - val_accuracy: 0.9986
Epoch 6/100
122461/122461 [==============================] - 127s 1ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.0134 - val_accuracy: 0.9985
Epoch 7/100
122461/122461 [==============================] - 126s 1ms/step - loss: 0.0012 

In [ ]:
t1 = datetime.datetime.now()
print(t1-t0)

0:27:39.455943


In [ ]:
from tensorflow.keras.models import load_model

classifier.save('KDD_ANN_model.h5')

In [ ]:
print("Accuracy: ", classifier.evaluate(X_train, y))

153076/153076 [==============================] - 101s 662us/step - loss: 0.0024 - accuracy: 0.9995
Accuracy:  [0.002422800287604332, 0.9995412826538086]


In [ ]:
df_test = pd.read_csv("./kddcup.testdata.unlabeled",header=None)
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0,udp,private,SF,105,146,0,0,0,0,...,1,1,1.0,0.00,1.00,0.0,0.0,0.0,0.0,0.0
1,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
2,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
3,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
4,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984154 entries, 0 to 2984153
Data columns (total 41 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       int64  
 1   1       object 
 2   2       object 
 3   3       object 
 4   4       int64  
 5   5       int64  
 6   6       int64  
 7   7       int64  
 8   8       int64  
 9   9       int64  
 10  10      int64  
 11  11      int64  
 12  12      int64  
 13  13      int64  
 14  14      int64  
 15  15      int64  
 16  16      int64  
 17  17      int64  
 18  18      int64  
 19  19      int64  
 20  20      int64  
 21  21      int64  
 22  22      int64  
 23  23      int64  
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      int64  
 32  32      int64  
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64
 40  

In [ ]:
X_test = df_test.values

In [ ]:
label_test_1 = LabelEncoder()
X_test[:, 1] = label_test_1.fit_transform(X_test[:,1])

label_test_2 = LabelEncoder()
X_test[:, 2] = label_test_2.fit_transform(X_test[:,2])

label_test_3 = LabelEncoder()
X_test[:, 3] = label_test_3.fit_transform(X_test[:,3])

X_test = ct.transform(X_test.tolist())

In [ ]:
X_test[:5]

array([[0.0, 0.0, 1.0, 0, 50, 9, 105, 146, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1, 1,
        1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, 0, 50, 9, 105, 146, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 255,
        254, 1.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, 0, 50, 9, 105, 146, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 255,
        254, 1.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, 0, 50, 9, 105, 146, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 255,
        254, 1.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, 0, 50, 9, 105, 146, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 255,
        254, 1.0, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0]], dty

In [ ]:
X_test = X_test[:, 1:]

In [ ]:
X_test = mmc.fit_transform(X_test)
X_test[:5]

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 7.24637681e-01,
        9.00000000e-01, 1.67129195e-06, 4.51764850e-06, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.95694716e-03,
        1.95694716e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.92156863e-03, 3.92156863e-03, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 7.24637681e-01,
        9.00000000e-01, 1.67129195e-06, 4.51764850e-06, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00,

In [ ]:
X_test.shape

(2984154, 42)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
y_pred[:5]

array([[3.8071227e-01],
       [3.3656238e-06],
       [3.3656238e-06],
       [3.3656238e-06],
       [3.5686182e-06]], dtype=float32)

In [ ]:
y_pred_round = y_pred.round(0)
y_pred_round

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
y_pred_inv = labelencoder_y.inverse_transform(y_pred_round.astype("int"))
y_pred_inv

array(['normal.', 'normal.', 'normal.', ..., 'normal.', 'normal.',
       'normal.'], dtype=object)

In [ ]:
unique, counts = np.unique(y_pred_inv, return_counts=True)

In [ ]:
display(unique, counts)

array(['normal.', 'others'], dtype=object)

array([ 724008, 2260146], dtype=int64)

* The unlabeled dataset is predicted to have 724008 normal intrusions and 2260146 abnormal intrusions, in which the abnormal is 3 times more than the normal. This ratio is similar to the labeled dataset's ratio (1:4).
* The model classifier has accuracy of 99.95% with loss equals to 0.0024. --> the model has a very high accuracy. This could be explained as the number of types of intrusions was narrowed from 25 classes to 2 classes, categorizing become easier as only one probability is needed to be calculated. 